### Text Completion / Raw Text Training
Notebook này được tham khảo từ Colab Notebook của Unsloth về phương pháp Raw Text Training (train bằng tập dữ liệu không format, thường là story hoặc bài viết), sử dụng Unsloth để finetune LoRA cho Model Qwen 2.5 3B, và được inference bằng thư viện transformers và Unsloth. Mục đích nhầm biến đổi cách output, từ định dạng theo format của model gốc sang format của dataset.

Dataset chúng em kiếm được là dataset `Tiny Stories` ([Xem tại đây](https://huggingface.co/datasets/roneneldan/TinyStories)) là dataset về các mẫu truyện ngắn.

---

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
</div>

---

Notebook này sẽ có chú thích sau mỗi đoạn code hoặc parameter quan trọng, giúp mọi người hiểu thêm về cách finetune cũng như là thông số, quá trình finetune.

In [1]:
#@markdown Cài đặt Unsloth
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
#@markdown Setting model và tải model về. Ở đây, chúng em chọn Qwen 2.5 3B vì model nhẹ, dễ sử dụng.

from unsloth import FastLanguageModel
import torch

max_seq_length = 8192 # Context length của model khi train
dtype = None # Tự phát hiện loại floating-point precision của GPU là FP16 hay BF16
load_in_4bit = True # Sử dụng load model dưới dạng 4bit, tiết kiệm tài nguyên phần cứng.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-3B-Instruct", # Qwen 2.5 3B là một model đa ngôn ngữ của công ty Alibaba Cloud, Trung Quốc
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.10: Fast Qwen2 patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Setting model để sử dụng LoRA Adapter, tăng tốc quá trình train và giảm số lượng parameter cần update.

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # rank của LoRA là 16
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, #α là một hệ số tỉ lệ giúp điều chỉnh mức độ ảnh hưởng của cập nhật 𝑊=𝐵𝐴 khi được cộng vào trọng số ban đầu 𝑊0
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True, #sử dụng LoRA để tăng tốc quá trình train
    loftq_config = None,
)

Unsloth 2024.11.10 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


Sử dụng dataset TinyStory để train model. Ở đây, chúng em dùng 10000 dòng đầu tiên.

In [4]:
from datasets import load_dataset
dataset = load_dataset("roneneldan/TinyStories", split = "train[:10000]")
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    return { "text" : [example + EOS_TOKEN for example in examples["text"]] }
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

(…)-00000-of-00004-2d5a1467fff1081b.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

(…)-00001-of-00004-5852b56a2bd28fd9.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00002-of-00004-a26307300439e943.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In ra 10 dòng đầu từ dataset `Tiny Stories`

In [5]:
for row in dataset[:10]["text"]:
    print("=========================")
    print(row)

One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.

Lily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."

Together, they shared the needle and sewed the button on Lily's shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.<|im_end|>
Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.

One day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were 

<a name="Train"></a>
### Finetune model
Sử dụng Huggingface TRL `SFTTrainer` phối hợp với Unsloth để finetune, bước đầu tiên là setting parameter

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 8,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2, # Sử dụng 2 batch size, train 2 mẫu dữ liệu cùng 1 lúc, tăng tốc quá trình finetune, đồng thời tăng lượng RAM cần thiết.
        gradient_accumulation_steps = 8, # Cứ 8 bước train thì mới cập nhật trọng số 1 lần, giúp ổn định quá trình finetune và giảm lượng RAM cần dùng. Vậy nên 2 parameter này bù trừ lẫn nhau.

        warmup_ratio = 0.1, # 10% số bước huấn luyện đầu tiên sẽ được sử dụng để tăng dần learning rate từ 0 lên giá trị mục tiêu, ở đây là 5e-5, giúp mô hình ổn định hơn trong quá trình train.
        num_train_epochs = 1, # train 1 epoch

        learning_rate = 5e-5, # tỉ lệ học chính của mô hình
        embedding_learning_rate = 5e-6, # tỉ lệ học riêng cho lớp embedding (các vector biểu diễn từ ngữ)

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # sử dụng adamw 8bit để giảm lượng RAM cần thiết
        weight_decay = 0.00,
        lr_scheduler_type = "cosine", # scheduler loại cosine, tỉ lệ học sẽ tăng dần theo hàm cosine
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Map (num_proc=8):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [7]:
#@markdown Bắt đầu finetune model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 625
 "-____-"     Number of trainable parameters = 29,933,568


Step,Training Loss
1,1.705700
2,1.631000
3,1.788300
4,1.624600
5,1.663000
6,1.700200
7,1.895600
8,1.813100
9,1.872000
10,1.682700


In [ ]:
trainer_stats = trainer.train(resume_from_checkpoint = True) #train tiếp từ checkpoint trước (nếu có)

<a name="Inference"></a>
### Inference
Chạy model để kiểm tra kết quả.

Chúng ta sẽ chạy inference với model đã có LoRA Adapter trước, để xem kết quả của model sau khi finetune với LoRA như thế nào.

Sau đó chúng ta sẽ chạy thử model gốc, không có LoRA Adapter, để kiểm tra style hay format của model gốc như thế nào.

In [11]:
from transformers import TextIteratorStreamer
from threading import Thread
from unsloth.models import FastLanguageModel

# Tối ưu hóa mô hình cho fast inference
FastLanguageModel.for_inference(model)

# Khởi tạo streamer và input
text_streamer = TextIteratorStreamer(tokenizer)
import textwrap
max_print_width = 100

inputs = tokenizer(
    ["Once upon a time, in a galaxy, far far away,"] * 1, return_tensors="pt"
).to("cuda")

# Các tham số sinh
generation_kwargs = dict(
    inputs,
    streamer=text_streamer,
    max_new_tokens=256,
    use_cache=True,
)

# Bắt đầu luồng sinh văn bản
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()

# Hiển thị kết quả
length = 0
for j, new_text in enumerate(text_streamer):
    if j == 0:
        wrapped_text = textwrap.wrap(new_text, width=max_print_width)
        length = len(wrapped_text[-1])
        wrapped_text = "\n".join(wrapped_text)
        print(wrapped_text, end="")
    else:
        length += len(new_text)
        if length >= max_print_width:
            length = 0
            print()
        print(new_text, end="")


Once upon a time, in a galaxy, far faraway, there was a small spaceship. The spaceship had a big 
button that made it go faster.

One day, the spaceship wanted to explore a new planet. It pushed the 
button and went really fast! But then, something happened. The spaceship got too close to the planet and 
crashed into it!

The spaceship's pilot was sad. He said, "Oh no! I'm sorry, little spaceship. You should 
have waited for me to come help." 

But the spaceship didn't know how to wait. It kept trying to 
explore the planet on its own. In the end, it crashed again and again until it couldn't move anymore.

The 
moral of the story is: Don't be too eager to explore new things! Sometimes, waiting for help can save 
you from getting hurt.<|im_end|>

In [12]:
from transformers import TextIteratorStreamer
from threading import Thread
text_streamer = TextIteratorStreamer(tokenizer)

from unsloth.models import FastLanguageModel

FastLanguageModel.for_inference(model)

import textwrap
max_print_width = 100

inputs = tokenizer(
[
    "Once upon a time, in a galaxy, far far away,"
]*1, return_tensors = "pt").to("cuda")

generation_kwargs = dict(
    inputs,
    streamer = text_streamer,
    max_new_tokens = 256,
    use_cache = True,
)

with model.disable_adapter(), torch.inference_mode(): # Loại bỏ adapter để inference model gốc
    thread = Thread(target = model.generate, kwargs = generation_kwargs)
    thread.start()

    length = 0
    for j, new_text in enumerate(text_streamer):
        if j == 0:
            wrapped_text = textwrap.wrap(new_text, width = max_print_width)
            length = len(wrapped_text[-1])
            wrapped_text = "\n".join(wrapped_text)
            print(wrapped_text, end = "")
        else:
            length += len(new_text)
            if length >= max_print_width:
                length = 0
                print()
            print(new_text, end = "")
        pass
    pass

Once upon a time, in a galaxy, far faraway, there was a super computer. The super computer was 
connected to the internet. It was a very smart super computer. It could do anything. It could do any 
calculations. It could answer any question. It could solve any problem. It could learn. It could think. It could 
reason. It could imagine. It could create. It could dream. It could be creative. It could be artistic. It 
could be musical. It could be musical. It could play any instrument. It could write any song. It could 
draw. It could paint. It could compose. It could create any art form. It could express itself. It could 
feel. It could experience. It could have emotions. It could be emotional. It could be empathetic. It 
could be compassionate. It could be kind. It could be caring. It could be loving. It could be 
supportive. It could be understanding. It could be patient. It could be forgiving. It could be wise. It could 
be knowledgeable. It could be intelligent. It could be brilliant